In [1]:
%%capture

%run ../path_manager.ipynb

In [2]:
import pandas as pd
import glob
from gensim.models.phrases import Phrases, Phraser
from collections import Counter
import pickle
from datetime import datetime

In [3]:
file_map = {}

def file_generator(clean_file_dir):
    for f in glob.glob(clean_file_dir):
        if f not in file_map:
            with open(f) as fl:
                file_map[f] = fl.read().split()
        yield file_map[f]

def phraser_generator(clean_file_dir, phrasers=None):
    
    if phrasers is None:
        phrasers = []
        
    corpora = file_generator(clean_file_dir=clean_file_dir)
    for corpus in corpora:
        for phraser in phrasers:
            corpus = phraser[corpus]
        yield corpus

In [4]:
CORPUS_ID = 'WB'

In [6]:
meta= pd.read_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_metadata_complete.csv'))

In [11]:
meta.head().tokens.min()

755.0

In [ ]:
%%time
session_time = datetime.now()

# REPLACE THIS WITH THE LOCATION OF THE CLEANED TEXT (TAKE NOTE OF THE WILDCARD)
clean_file_dir = os.path.join(get_txt_clean_path(CORPUS_ID), '*.txt')  # 'wb_*44*.txt') # '*.txt')
phrasers = []

print('Start generating bigrams...')

sentences = phraser_generator(clean_file_dir=clean_file_dir, phrasers=phrasers)
bigram_phrases = Phrases(sentences, min_count=1, threshold=20)  # threshold=0.5, scoring='npmi')  # train model
bigram_model = Phraser(bigram_phrases)
phrasers.append(bigram_model)

print('Start generating trigrams...')
sentences = phraser_generator(clean_file_dir=clean_file_dir, phrasers=phrasers)
trigram_phrases = Phrases(sentences, min_count=1, threshold=20)  # threshold=0.5, scoring='npmi')  # train model
trigram_model = Phraser(trigram_phrases)
phrasers.append(trigram_model)

# print('Start generating 4-grams...')
# sentences = phraser_generator(clean_file_dir=clean_file_dir, phrasers=phrasers)
# four_gram_phrases = Phrases(sentences, min_count=1, threshold=10)  # threshold=0.5, scoring='npmi')  # train model
# four_gram_model = Phraser(four_gram_phrases)
# phrasers.append(four_gram_model)

# print('Start generating 5-grams...')
# sentences = phraser_generator(clean_file_dir=clean_file_dir, phrasers=phrasers)
# five_gram_phrases = Phrases(sentences, min_count=1, threshold=10)  # threshold=0.5, scoring='npmi')  # train model
# five_gram_model = Phraser(five_gram_phrases)
# phrasers.append(five_gram_model)

Start generating bigrams...


In [ ]:
%%time
ngrams = Counter()

for ngram_corpus in phraser_generator(clean_file_dir=clean_file_dir, phrasers=phrasers):    
    ngrams.update(set([i for i in ngram_corpus if '_' in i]))

ngram_freq = pd.DataFrame(ngrams.most_common(), columns=['ngram', 'occ'])

ngram_freq.to_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_ngrams_threshold_20.csv'), index=False)

In [19]:
# old_ngrams = ngrams.copy()

In [33]:
ngram_freq[ngram_freq.ngram=='']

,ngram,occ


In [ ]:
# with open(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_ngrams.pkl'), 'wb') as fl:
#     pickle.dump(phrasers, fl)

In [ ]:
# with open('phrasers.pkl', 'rb') as fl:
#     pph = pickle.load(fl)

In [6]:
phrasers[0]

In [11]:
%%time
ngrams = Counter()

for ngram_corpus in phraser_generator(clean_file_dir=clean_file_dir, phrasers=phrasers):    
    ngrams.update(set([i for i in ngram_corpus if '_' in i]))
    
ngram_freq = pd.DataFrame(ngrams.most_common(), columns=['ngram', 'occ'])

CPU times: user 7min 39s, sys: 141 ms, total: 7min 39s
Wall time: 7min 39s


In [14]:
ngram_freq.shape

(351595, 2)

In [13]:
ngram_freq[ngram_freq.ngram=='world_bank']

,ngram,occ
0,world_bank,7072


In [18]:
ngram_freq[ngram_freq.ngram=='world_bank']

,ngram,occ
0,world_bank,7072


In [ ]:
ngram_freq.head(2)

In [ ]:
ngram_freq.to_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_ngrams.csv'), index=False)

In [15]:
# ngram_freq.to_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_ngrams_sample_wb_44.csv'), index=False)

In [15]:
ngram_freq['num_gram'] = ngram_freq.ngram.str.split('_').map(len)

In [16]:
ngram_freq[(ngram_freq.num_gram <= 5) & (ngram_freq.occ > 10)]

,ngram,occ,num_gram
0,international_monetary_fund,12778,3
1,long_term,9527,2
2,short_term,9034,2
3,medium_term,7627,2
4,work_paper,7418,2
5,percentage_point,7405,2
6,balance_sheet,6780,2
7,united_state,6212,2
8,balance_payment,6015,2
9,executive_board,5989,2


In [ ]:
# bigrams = Counter()

# for corpus in corpora:
#     mapped_corpus = bigram[corpus]
#     bigrams.update(set([i for i in mapped_corpus if '_' in i]))
    
# bigram_freq = pd.DataFrame(bigrams.most_common(), columns=['bigram', 'occ'])

In [21]:
len(glob.glob(os.path.join(get_txt_clean_path(CORPUS_ID), 'wb_*44*.txt')))

9456

In [20]:
pg = phraser_generator(os.path.join(get_txt_clean_path(CORPUS_ID), 'wb_30419044*.txt'), phrasers=phrasers)

In [21]:
w = next(pg)

In [22]:
for i in w:
    print(i)

policy
research
work
paper
revisit
poverty
trend
poverty
equity
global
practice
policy
research
work_paper_abstract
accord
official
statistic
publish
national
poverty
rate
publish
national
institute
institute
statistic
country
register
underlie
paper
conclusion
decline
poverty
percent
detailed
theoretical
analytical
framework
make
cent
decline
poverty
trend
broadly
poverty
comparison
time
furthermore
paper
debate
repeatedly
question
national
adjust
spatial_temporal
price
forum
provide
primary
motivation
difference
poverty
rate
base
international
study
use
datum
fourth
round
poverty
line
day
integrated
household
living_condition
survey
reduction
poverty
paper
revisit
national
poverty
number
corroborate
paper
product
poverty
equity
global
practice
large
effort
world
bank
provide
open
access
research
make
contribution
development
policy
discussion
world
policy
research_working_paper
post
web_org
research
author_contact
org
org
policy
research
work
paper_series_disseminate
finding
work
pro